## Домашнее задание 2.

## Задание 1.

**Задание**: обучите три классификатора: 

1) на токенах с высокой частотой 

2) на токенах со средней частотой 

3) на токенах с низкой частотой

Сравните полученные результаты, оцените какие токены наиболее важные для классификации.

## Решение:

Загрузим библиотеки необходимые для дальнейшего обучения.

In [1]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 34.4 MB 1.3 MB/s 
     |████████████████████████████████| 41 kB 158 kB/s 
     |████████████████████████████████| 49 kB 8.0 MB/s 
     |████████████████████████████████| 55 kB 5.1 MB/s 
     |████████████████████████████████| 8.2 MB 49.4 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=4b51ed3b1526cecc049a7827e754b5e10813405ce4799e021a8a421ff0f39c13
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=97f102708f7f48882289803ea9e9604b04dd803f2b43718fa3e586fec125fa2a
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built intervaltree docopt
  Attempting uninstall: intervaltree
    Fou

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from collections import Counter
from nltk.corpus import stopwords
from string import punctuation
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger

Создадим функцию для лемматизации текста на основе библиотеки Natasha.

In [5]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def natasha_lemmatize(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return " ".join(_.lemma for _ in doc.tokens)

Сформируем общий датасет на основе наборов для обучения и теста.

In [6]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.reset_index(inplace=True)

In [7]:
df['text'][1]

'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'

In [8]:
natasha_lemmatize(df['text'][1])

'да , все-таки он немного похожий на он . но мой мальчик весь равный лучше : d'

Проведем лемматизацию твитов в датасете.

In [9]:
df['lemmatized_text'] = df['text'].apply(natasha_lemmatize)
df.head()

,index,text,label,lemmatized_text
0,0,"@first_timee хоть я и школота, но поверь, у на...",positive,"@ first_timee хоть я и школоть , но поверь , у..."
1,1,"Да, все-таки он немного похож на него. Но мой ...",positive,"да , все-таки он немного похожий на он . но мо..."
2,2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt @ katiacheh : ну ты идиотка ) я испугаться ...
3,3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,"rt @ digger 2912 : "" кто тот в угол сидеть и п..."
4,4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,@ irina_dyshkant вот что значить страшилка : d...


Создадим корпус используемых в твитах слов.

In [108]:
noise = stopwords.words('russian') + list(punctuation)
corpus = [token for tweet in df['lemmatized_text'] 
          for token in nltk.tokenize.word_tokenize(tweet) if token not in noise]
print(len(corpus))
corpus[:10]

2059377


['first_timee',
 'школоть',
 'поверь',
 'самый',
 'd',
 'общество',
 'профилировать',
 'предмет',
 'тип',
 'все-таки']

Посмотрим на самые часто встречающиеся слова.

In [109]:
freq_dict = Counter(corpus)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
list(freq_dict_sorted)[:10]

[('rt', 38132),
 ('t', 33881),
 ('http', 33072),
 ('co', 33063),
 ('весь', 29865),
 ('``', 23933),
 ('это', 22598),
 ('...', 22363),
 ('d', 18954),
 ('хотеть', 11790)]

Выделим три вида токенов в зависимости от частоты их появления в твитах и обучим логистическую регрессию на основе каждого вида.

In [110]:
most_frequent_tokens = [item[0] for item in freq_dict.most_common(90000)][:30000]
avg_frequent_tokens = [item[0] for item in freq_dict.most_common(90000)][30000:60000]
less_frequent_tokens = [item[0] for item in freq_dict.most_common(90000)][60000:]
most_frequent_tokens[:10]

['rt', 't', 'http', 'co', 'весь', '``', 'это', '...', 'd', 'хотеть']

Разделим датасет на тренировчную и валидационную части.

In [111]:
x_train, x_test, y_train, y_test = train_test_split(df['lemmatized_text'], df.label, random_state=42)

Классификатор на токенах с самой высокой частотой.

In [112]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.78      0.75      0.77     29043
    positive       0.75      0.78      0.76     27666

    accuracy                           0.76     56709
   macro avg       0.77      0.77      0.76     56709
weighted avg       0.77      0.76      0.77     56709



Классификатор на токенах со средней частотой.

In [113]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=avg_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.92      0.53      0.67     48686
    positive       0.20      0.71      0.31      8023

    accuracy                           0.55     56709
   macro avg       0.56      0.62      0.49     56709
weighted avg       0.81      0.55      0.62     56709



Классификатор на токенах с низкой частотой.

In [114]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=less_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.08      0.72      0.15      3230
    positive       0.97      0.52      0.68     53479

    accuracy                           0.53     56709
   macro avg       0.53      0.62      0.41     56709
weighted avg       0.92      0.53      0.65     56709



Как видим наилучший результат классификатора получается при обучении на самых часто встречаемых токенах. На мой взгляд это логично, так как благодаря работе с наиболее часто встречающимися токенами мы получаем больший датасет для обучения модели.

## Задание 2.

найти фичи с наибольшей значимостью, и вывести их

## Решение:

Вернемся к классификатору на самых частотных токенах как к показавшему наилучшие результаты и определим наиболее значимые признаки.

In [115]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.78      0.75      0.77     29043
    positive       0.75      0.78      0.76     27666

    accuracy                           0.76     56709
   macro avg       0.77      0.77      0.76     56709
weighted avg       0.77      0.76      0.77     56709



Напечатаем наиболее значимые токены. 

In [116]:
coefs = clf.coef_[0]
top_coefs = np.argsort(coefs)[-20:]
top_coefs

array([ 5354,  3906, 10438,  1019,  2493,  8889,  5356,  4752,  3978,
       10521,  8485, 18192, 18135,  1482,  2576,  1369,     8,  1161,
         399,   154])

In [117]:
tokens = list(vec.vocabulary_.items())
for _ in reversed(top_coefs):
    print(tokens[_])

('dd', 154)
('ddd', 399)
('dddd', 1161)
('d', 8)
('xd', 1369)
('ddddd', 2576)
('ахахи', 1482)
('zaharikov', 18135)
('запарен', 18192)
('эротический', 8485)
('dddddddd', 10521)
('juljulianapai', 3978)
('jaredleto', 4752)
('dddddd', 5356)
('листаться', 8889)
('ржа', 2493)
('хахах', 1019)
('iron_wings', 10438)
('спасииибо', 3906)
('постепенно', 5354)


Как видим наиболее значимыми оказываются токены, которые получились после преобразования смайлов, а также те, которые имеют явно выраженную эмоциональную окраску ("ржа", "спасииибо", "запарен").

### Задание 3.

1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера 

3) убедиться что для сетки нет переобучения

## Решение:

Построим модели на основе других векторайзеров и сравним получившиеся результаты.

In [119]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.76      0.76      0.76     28004
    positive       0.77      0.76      0.76     28705

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



In [120]:
features_num = [2**19, 2**20, 2**21, 2**22]

for n_features in features_num:
    print(f'Number of features - {n_features}:')
    vec = HashingVectorizer(n_features=n_features,)
    bow = vec.fit_transform(x_train)
    clf = LogisticRegression(random_state=42, max_iter=500)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))
    print(classification_report(pred, y_test))

Number of features - 524288:
              precision    recall  f1-score   support

    negative       0.72      0.76      0.74     26921
    positive       0.77      0.74      0.75     29788

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

Number of features - 1048576:
              precision    recall  f1-score   support

    negative       0.72      0.76      0.74     26935
    positive       0.77      0.74      0.76     29774

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709

Number of features - 2097152:
              precision    recall  f1-score   support

    negative       0.72      0.76      0.74     26954
    positive       0.77      0.74      0.75     29755

    accuracy                           0.75     56709
   macro avg       0.75      0.75      

Оптимальное количество фичей в модели с HashingVectorizer - *1048576*.

In [121]:
vec = HashingVectorizer(n_features=2**19,)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.72      0.76      0.74     26921
    positive       0.77      0.74      0.75     29788

    accuracy                           0.75     56709
   macro avg       0.75      0.75      0.75     56709
weighted avg       0.75      0.75      0.75     56709



Теперь обучим нейронную сеть.

In [164]:
y_train_bin = np.where(y_train=='positive', 1, 0)
y_test_bin = np.where(y_test=='positive', 1, 0)

In [165]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train_bin))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test_bin))

In [166]:
train_data = train_data.batch(32)
valid_data = valid_data.batch(32)

In [167]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [168]:
def custom_standardization(input_data):
    
    return input_data

vocab_size = 15000
seq_len = 140

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [273]:
embedding_dim=250

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(1)
])

In [274]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [275]:
model.fit(train_data, validation_data=valid_data, epochs=2)

Epoch 1/2
5317/5317 [==============================] - 114s 21ms/step - loss: 0.1829 - accuracy: 0.9216 - val_loss: 0.2963 - val_accuracy: 0.8497
Epoch 2/2
5317/5317 [==============================] - 108s 20ms/step - loss: 0.2361 - accuracy: 0.9007 - val_loss: 0.1894 - val_accuracy: 0.9173


In [276]:
logits = model.predict(valid_data)
prediction = tf.round(tf.nn.sigmoid(logits))
class_preds = np.where(prediction==1, 'positive', 'negative')

Построим для данной модели матрицу ошибок.

In [278]:
print(classification_report(class_preds, y_test))

              precision    recall  f1-score   support

    negative       0.93      0.93      0.93     28191
    positive       0.93      0.93      0.93     28518

    accuracy                           0.93     56709
   macro avg       0.93      0.93      0.93     56709
weighted avg       0.93      0.93      0.93     56709



Как видим данная нейросеть показала самые лучшие результаты по сравнению с остальными моделями.